# Model training
Using a neural network for the recommender system

In [3]:
import pandas as pd

In [ ]:
activites = pd.read_csv('activities.csv')
activites.head()

In [6]:
# Read the CSV of interactions and sort it by timestamp
data = pd.read_csv('interactions.csv')
data_sorted = data.sort_values('timestamp')
data_sorted.head()

,user_id,item_id,rating,timestamp
0,1,0,2,1616193271
2,1,2,0,1616193911
1,1,1,1,1616194035
3,1,3,0,1616194210
4,0,2,2,1616194210


In [4]:
# Cut the data off at 75% in order to save some data for the test process
CUTOFF = 0.75
cutoff_idx = int(len(data_sorted) * CUTOFF)

# Generate the train and test data
data_train = data_sorted.iloc[0:cutoff_idx]
data_test = data_sorted.iloc[cutoff_idx:]

data_train.shape, data_test.shape

((3, 8), (2, 8))

In [ ]:
def get_intersection_test_and_train(field):
    return (
        set(df_test[field].unique())
        & set(df_train[field].unique())
    )

data_test_clean = (
    data_test.loc[
        data_test["user"]
    ]
)